In [ ]:
# streamlit_app.py
import streamlit as st
import pandas as pd
import numpy as np
import pickle
import os

st.set_page_config(page_title="Car Price Predictor", layout="centered")

# Load model objects
MODEL_PATH = "models/best_model_pickle.pkl"
SCALER_PATH = "models/scaler_pickle.pkl"
FEATURES_PATH = "models/model_features_pickle.pkl"

st.title("🚗 Car Price Prediction")
st.markdown("Enter car details and get an estimated selling price.")

# --- INPUTS (adjust labels to match YOUR selected_features) ---
st.sidebar.header("Input features")
vehicle_age = st.sidebar.number_input("Vehicle age (years)", min_value=0, max_value=50, value=5)
km_driven = st.sidebar.number_input("Kilometres driven", min_value=0, value=20000)
mileage = st.sidebar.number_input("Mileage (kmpl)", min_value=0.0, value=18.0, format="%.2f")
engine = st.sidebar.number_input("Engine (cc)", min_value=500, value=1197)
max_power = st.sidebar.number_input("Max power", min_value=10.0, value=80.0, format="%.2f")
seats = st.sidebar.number_input("Seats", min_value=1, max_value=10, value=5)

brand = st.sidebar.selectbox("Brand", ["Maruti", "Hyundai", "Toyota", "Mahindra", "Honda", "Ford", "Other"])
fuel_type = st.sidebar.selectbox("Fuel type", ["Petrol", "Diesel", "CNG", "LPG", "Electric"])
transmission = st.sidebar.selectbox("Transmission", ["Manual", "Automatic"])
seller_type = st.sidebar.selectbox("Seller type", ["Individual", "Dealer"])

# Build a DataFrame representing a single example
row = {
    "vehicle_age": vehicle_age,
    "km_driven": km_driven,
    "mileage": mileage,
    "engine": engine,
    "max_power": max_power,
    "seats": seats
}

# Create DataFrame
input_df = pd.DataFrame([row])

# Add categorical dummy variables (0 by default)
# Get all feature names that look like dummies for categories
for feat in model_features:
    if feat not in input_df.columns:
        input_df[feat] = 0

# Map simple categories to dummy names (adjust prefix to match your training -> e.g., 'brand_Maruti')
brand_col = f"brand_{brand}"
if brand_col in input_df.columns:
    input_df[brand_col] = 1

fuel_col = f"fuel_type_{fuel_type}"
if fuel_col in input_df.columns:
    input_df[fuel_col] = 1

trans_col = f"transmission_type_{transmission}"
if trans_col in input_df.columns:
    input_df[trans_col] = 1

seller_col = f"seller_type_{seller_type}"
if seller_col in input_df.columns:
    input_df[seller_col] = 1

# Ensure column order matches the model_features list
input_df = input_df.reindex(columns=model_features, fill_value=0)

# Scale numeric columns using loaded scaler
X_scaled = scaler.transform(input_df)

st.subheader("Inputs (processed)")
st.write(input_df)

if st.button("Predict selling price"):
    pred = model.predict(X_scaled)[0]
    st.success(f"Estimated selling price: {pred:,.0f}")
